# Visual odometry pipeline

### Import requirements

In [1]:
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np

from ImageProcessor import ImageProcessor
from helperScripts.TimeKeeper import TimeKeeper

%matplotlib ipympl

### Load images and depth map

In [2]:
folderChoice = 0
path = "".join(["testImages/visualOdometryTestImages/", str(folderChoice)])

# Load set of top images
imageGlobT = sorted(glob.glob("".join([path, "/top_*", ".png"])))

# Load set of bottom images
imageGlobB = sorted(glob.glob("".join([path, "/bottom_*", ".png"])))

# Load depth map
imageGlobD = sorted(glob.glob("".join([path, "/topDepth_*", ".png"])))

if not (len(imageGlobT)==len(imageGlobB) and \
                len(imageGlobB)==len(imageGlobD)):
    print("Images could not be matched")

print ("Selections: 0-{}".format(len(imageGlobT)-1))

Selections: 0-1


In [3]:
# Top images
imagesT = []
for imageFile in imageGlobT:
    imagesT.append(cv2.imread(imageFile))

# Bottom images
imagesB = []
for imageFile in imageGlobB:
    imagesB.append(cv2.imread(imageFile))

# Depth maps; -1 flag to load them as is
depthMaps = []
for imageFile in imageGlobD:
    depthMaps.append(cv2.imread(imageFile, -1))

print(len(imagesT), len(imagesB), len(depthMaps))

2 2 2


### View top camera images

In [4]:
plt.figure()
plt.suptitle("Top images")
plt.imshow(cv2.cvtColor(np.hstack(imagesT), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### View bottom camera images

In [5]:
plt.figure()
plt.suptitle("Bottom images")
plt.imshow(cv2.cvtColor(np.hstack(imagesB), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### View depth images
Depth images correspond to top camera

In [6]:
plt.figure()
plt.suptitle("Depth maps")
plt.imshow(np.hstack(depthMaps))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Depth map units and dimensions

In [7]:
print(depthMaps[0].shape)
print(depthMaps[0][int(depthMaps[0].shape[0]/2), \
                    int(depthMaps[0].shape[1]/2)], "mm")

(640, 360)
750 mm


### Instantiating TimeKeeper

In [8]:
timekeeper = TimeKeeper()

### Loading camera calibration matrix

In [9]:
imageProcessor=ImageProcessor()
imageProcessor.loadMonoCalibration()
k=imageProcessor.cameraMatrixL
print(k)

Reading from data/monoCalibration.json
Loaded mono calibration
[[592.63974229   0.         188.47568825]
 [  0.         592.98181459 312.47351841]
 [  0.           0.           1.        ]]


### Feature Extraction


In [10]:
def extractFeatures(orb, image):
    """Find keypoins and descriptors for the image"""
    keypoints = orb.detect(image, None)
    keypoints, descriptors = orb.compute(image, keypoints)
    
    return keypoints, descriptors

In [11]:
selection = 0

orb = cv2.ORB_create()

keypoints, descriptors = extractFeatures(orb, imagesT[selection])

print("Number of features detected in frame {}: {}"\
                                .format(selection, len(keypoints)))
print("Coordinates of first keypoint in frame {}: {}"\
                                .format(selection, str(keypoints[0].pt)))


Number of features detected in frame 0: 500
Coordinates of first keypoint in frame 0: (299.0, 596.0)


### Visualize features

In [12]:
def visualizeFeatures(image, keypoints, flag):
    """Visualize extracted features in image"""
    display = cv2.drawKeypoints(image, keypoints, None, flags=flag)
    plt.figure(figsize=(7, 7))
    plt.imshow(cv2.cvtColor(display, cv2.COLOR_BGR2RGB))

In [13]:
visualizeFeatures(imagesT[selection], keypoints, 4)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
visualizeFeatures(imagesT[selection], keypoints, 2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
def extractAllFeatures(images, extractFeatures, orb):
    """Find keypoints and descriptors for each image in folder"""
    allKeypoints = []
    allDescriptors = []
    
    for image in images:
        keypoints, descriptors = extractFeatures(orb, image)
        allKeypoints.append(keypoints)
        allDescriptors.append(descriptors)
    
    return allKeypoints, allDescriptors

In [16]:
allKeypoints, allDescriptors = extractAllFeatures(imagesT, extractFeatures, orb)

print(len(allKeypoints), len(allDescriptors))

print("Number of features detected in frame {}: {}"\
                            .format(selection, len(allKeypoints[selection])))
print("Coordinates of the first keypoint in frame {}: {}\n"\
                            .format(selection, str(allKeypoints[selection][0].pt)))

2 2
Number of features detected in frame 0: 500
Coordinates of the first keypoint in frame 0: (299.0, 596.0)



### Feature matching

In [17]:
def matchFeatures(bfMatcher, descriptors0, descriptors1, bestNMatches):
    """Match features from two images"""
    match = bfMatcher.match(descriptors0, descriptors1)
    match = sorted(match, key = lambda x:x.distance)

    return match[:bestNMatches]

In [18]:
bfMatcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

bestNMatches = 50

match = matchFeatures(bfMatcher, allDescriptors[selection], \
                    allDescriptors[selection+1], bestNMatches)

print("Number of features matched in frames {} and {}: {}"\
                        .format(selection, selection+1, len(match)))

Number of features matched in frames 0 and 1: 50


In [19]:
def visualizeMatches(image0, keypoints0, image1, keypoints1, match):
    imageMatches = cv2.drawMatches(image0, keypoints0, \
                            image1, keypoints1, match, None, flags=2)

    plt.figure(figsize=(7, 7))
    plt.imshow(cv2.cvtColor(imageMatches, cv2.COLOR_BGR2RGB))

In [20]:
visualizeMatches(imagesT[selection], allKeypoints[selection], \
                imagesT[selection+1], allKeypoints[selection+1], match)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
def matchAllFeatures(allDescriptors, matchFeatures, bfMatcher, bestNMatches):
    """Match features for each subsequent image pair in the dataset"""
    matches = []

    for i in range(len(allDescriptors)-1):
        descriptor1 = allDescriptors[i]
        descriptor2 = allDescriptors[i+1]

        match = matchFeatures(bfMatcher, descriptor1, descriptor2, bestNMatches)

        matches.append(match)
    
    return matches

In [22]:
matches = matchAllFeatures(allDescriptors, matchFeatures, bfMatcher, bestNMatches)

print("Number of features matched in frames {} and {}: {}"\
            .format(selection, selection+1, len(matches[selection])))

Number of features matched in frames 0 and 1: 50


### Trajectory Estimation

In [23]:
def estimateMotion(match, keypoints0, keypoints1, k, depthMap=None):
    """Estimate camera motion from a pair of subsequent image frames"""
    rmat=np.eye(3)
    tvec=np.zeros((3,1))
    imagePoints0 = []
    imagePoints1 = []

    for m in match:
        train_idx = m.trainIdx
        query_idx = m.queryIdx

        p1x, p1y = keypoints0[query_idx].pt 
        imagePoints0.append([p1x,p1y])

        p2x,p2y = keypoints1[train_idx].pt 
        imagePoints1.append([p2x,p2y])

    E, mask = cv2.findEssentialMat(\
                        np.array(imagePoints0), np.array(imagePoints1), k) 

    retval, rmat, tvec, mask = cv2.recoverPose(\
                        E, np.array(imagePoints0), np.array(imagePoints1), k)

    return rmat, tvec, imagePoints0, imagePoints1

In [24]:
match = matches[selection]
rmat, tvec, imagePoints0, imagePoints1 = estimateMotion(\
                        match, allKeypoints[selection], \
                        allKeypoints[selection+1], k, \
                        depthMap=depthMaps[selection])

print("Estimated rotation:\n {0}".format(rmat))
print("Estimated translation:\n {0}".format(tvec))

Estimated rotation:
 [[ 0.99721259 -0.0275397   0.06934423]
 [ 0.03194127  0.99749043 -0.06318699]
 [-0.06743006  0.06522581  0.99558966]]
Estimated translation:
 [[-0.00686333]
 [-0.23818576]
 [-0.97119537]]


### Camera movement visualization

In [25]:
def visualizeCameraMovement(image0, imagePoints0, \
                image1, imagePoints1, showImageAfterMove=False):
    """Visualize camera movement across frames"""
    image0 = image0.copy()
    image1 = image1.copy()

    for i in range(0, len(imagePoints0)):
        # Coordinates of a point on t frame
        p1 = (int(imagePoints0[i][0]), int(imagePoints0[i][1]))
        # Coordinates of the same point on t+1 frame
        p2 = (int(imagePoints1[i][0]), int(imagePoints1[i][1]))

        cv2.circle(image0, p1, 5, (0, 255, 0), 1)
        cv2.arrowedLine(image0, p1, p2, (0, 255, 0), 1)
        cv2.circle(image0, p2, 5, (255, 0, 0), 1)

        if showImageAfterMove:
            cv2.circle(image1, p2, 5, (255, 0, 0), 1)
    
    if showImageAfterMove: 
        return image1
    else:
        return image0

In [26]:
imageMovementBefore = visualizeCameraMovement(imagesT[selection], \
                    imagePoints0, imagesT[selection+1], imagePoints1)

imageMovementAfter = visualizeCameraMovement(imagesT[selection], \
                    imagePoints0, imagesT[selection+1], imagePoints1, \
                    showImageAfterMove=True)

plt.figure(figsize=(7,7))
plt.imshow(cv2.cvtColor(np.hstack([imageMovementBefore, imageMovementAfter]), \
                                                        cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Camera Trajectory Estimation

In [27]:
def estimateTrajectory(estimateMotion, matches, allKeypoints, k, depthMaps):
    """Estimate complete camera trajectory from subsequent image pairs"""
    trajectory = [np.array([0, 0, 0])]

    R = np.diag([1,1,1])
    T = np.zeros([3, 1])
    RT = np.hstack([R, T])
    RT = np.vstack([RT, np.zeros([1, 4])])
    RT[-1, -1] = 1

    for i in range(len(matches)):     
        match = matches[i]
        keypoints0 = allKeypoints[i]
        keypoints1 = allKeypoints[i+1]
        depth = depthMaps[i]

        rmat, tvec, imagePoints0, imagePoints1 = estimateMotion(\
                                match, keypoints0, keypoints1, k, depthMap=depth)

        rt_mtx = np.hstack([rmat, tvec])
        rt_mtx = np.vstack([rt_mtx, np.zeros([1, 4])])
        rt_mtx[-1, -1] = 1

        rt_mtx_inv = np.linalg.inv(rt_mtx)
        
        RT = np.dot(RT, rt_mtx_inv)
        newTrajectory = RT[:3, 3]
        trajectory.append(newTrajectory)

    trajectory = np.array(trajectory).T 

    return trajectory 

In [28]:
trajectory = estimateTrajectory(estimateMotion, matches, \
                            allKeypoints, k, depthMaps=depthMaps)

print("Camera location in point {} is: \n {}\n"\
                        .format(selection, trajectory[:, [selection]]))
print("Camera location in point {} is: \n {}\n"\
                        .format(selection+1, trajectory[:, [selection+1]]))
print("Length of trajectory: {}".format(trajectory.shape[1]))

Camera location in point 0 is: 
 [[0.]
 [0.]
 [0.]]

Camera location in point 1 is: 
 [[-0.0510356 ]
 [ 0.30074601]
 [ 0.95233776]]

Length of trajectory: 2
